test1　　演算(+,*)のテスト

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [3]:
%cd "ctorch210701/MiniCTorch_Prototype"

/content/drive/My Drive/Colab Notebooks/ctorch210701/MiniCTorch_Prototype


In [4]:
!pwd

/content/drive/My Drive/Colab Notebooks/ctorch210701/MiniCTorch_Prototype


In [5]:
! pip install lark-parser

In [6]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch
import minictorch.generator as GN
import minictorch.converter as CV

In [7]:
class Net(nn.Module):
   def __init__(self):
      super(Net, self).__init__()
      self.t = torch.tensor(np.array([[1.0, 2], [3, 4]]))
           
      # epsilon = torchrandn.(mean.shape).to(device)
      # return mean + epsilon

   def forward(self, x):
      f1 = 10 * x * x * self.t
      f2 = 5 * x
      y = f1 + f2
      return y
      #return F.softmax( input_x , dim=1)

In [21]:
def generate_json( project, input_x, json_path, check=1 ):

   model = Net()
   #model = torch.nn.LogSoftmax()

   model.eval()
   with torch.no_grad():
      GN.generate_minictorch_file( model, input_x, json_path )
    
   if check > 0:
      yy = model( input_x )
      print(yy)
      yy.sum().backward()
      print(input_x.grad)

   return model

In [22]:
def convert_json( project, model, input_x, json_path, rand_flag=0 ):

    folder = "src"
    cpp_fname   = project + ".cpp"
    param_fname = project + "_param.h"
    cpp_path    = folder + "/" + cpp_fname
    param_path  = folder + "/" + param_fname
    make_path   = folder + "/" + "Makefile"
  
    # save json file
    print( "[JSON]", json_path )
    fp = open( json_path )
    obj = json.load( fp )

    # save parameter file
    code1 = CV.c_param_generator( obj, model, input_x )
    if len(code1) > 0:
       print( "[PARAM]", param_path )
       ofparam = open( param_path, "w" )
       ofparam.write( code1 )
    else:
       param_fname=""

    #save cpp file
    code2 = CV.c_code_generator( obj, model, param_fname, rand_flag )

    print("[CPP] ", cpp_path )
    ofp = open( cpp_path, "w" )
    ofp.write( code2 )

    # save make file
    make_code = CV.makefile_generator( cpp_fname )

    print( "[MAKE]", make_path )
    makefp = open( make_path, "w" )
    makefp.write( make_code )

In [23]:
input_to_model = torch.tensor( np.array([[1.0, 2], [3, 4]]), requires_grad=True)
#input_to_model = torch.randn(1,5,requires_grad=True)
print(input_to_model)

project = "test1"
json_fname = project + ".json"
json_path  = "network/" + json_fname
model = generate_json( project, input_to_model, json_path )

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64, requires_grad=True)
tensor([[ 15.,  90.],
        [285., 660.]], dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([[ 25.,  85.],
        [185., 325.]], dtype=torch.float64)


In [24]:
convert_json( project, model, input_to_model, json_path, 0 )

[JSON] network/test1.json
[PARAM] src/test1_param.h
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'shape': [2, 2], 'out': [3, 2, 7], 'sorted_id': 0}
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'shape': [], 'constant_value': 10.0, 'out': [2], 'sorted_id': 1}
{'name': 'Net/5', 'op': 'aten::mul', 'in': [0, 1], 'shape': [2, 2], 'out': [3], 'sorted_id': 2}
{'name': 'Net/6', 'op': 'aten::mul', 'in': [2, 0], 'shape': [2, 2], 'out': [5], 'sorted_id': 3}
{'name': 'Net/7', 'op': 'prim::Constant', 'in': [], 'shape': [2, 2], 'constant_value': [1.0, 2.0, 3.0, 4.0], 'out': [5], 'sorted_id': 4}
{'name': 'Net/f1', 'op': 'aten::mul', 'in': [3, 4], 'shape': [2, 2], 'out': [9], 'sorted_id': 5}
{'name': 'Net/9', 'op': 'prim::Constant', 'in': [], 'shape': [], 'constant_value': 5.0, 'out': [7], 'sorted_id': 6}
{'name': 'Net/f2', 'op': 'aten::mul', 'in': [0, 6], 'shape': [2, 2], 'out': [9], 'sorted_id': 7}
{'name': 'Net/11', 'op': 'prim::Constant', 'in': [], 'shape': [], 'constant_value': 1.0, 'out'

In [27]:
!g++ -std=c++14 ./src/test1.cpp -I ../../ctorch/lib -lcblas -o ./test1

(注意)  ctorch/libにxtensor関連のincludeを置いています。

In [26]:
!./test1

### forward computation ...
{{  15.,   90.},
 { 285.,  660.}}
### backward computation ...
{{  25.,   85.},
 { 185.,  325.}}
